#### Step 1: Load Cases are combined and saved as Parquet file (all_cs_loadcases.parquet). Convert it into a dataframe.

In [4]:
# Import relevant libraries
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import reduce

In [5]:
# Location of all_cs_loadcases
all_cs_loadcases = 'C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\all_cs_loadcases.parquet'

# Convert the combined Load Case parquet file to a dataframe
all_cs_loadcases = pd.read_parquet(all_cs_loadcases)

In [6]:
# Reduce demicals point to a hundredth place
cols_to_reduce = ['P', 'V2', 'V3', 'T', 'M2','M3']

all_cs_loadcases[cols_to_reduce] = all_cs_loadcases[cols_to_reduce].round(2)

#### Step 2: Break the DataFrame. Each smaller df contains different force types.

In [7]:
all_cs_loadcases.columns

Index(['Elm', 'ElmSta', 'LoadCase', 'StepNum', 'P', 'V2', 'V3', 'T', 'M2',
       'M3'],
      dtype='object')

In [8]:
all_cs_loadcases.head()

,Elm,ElmSta,LoadCase,StepNum,P,V2,V3,T,M2,M3
0,14253,0.0,Egrav6_0.2SUB,None,3.51,-30.47,-0.72,0.00,0.00,0.00
1,14253,1.5,Egrav6_0.2SUB,None,3.51,-28.17,-0.72,0.00,1.08,44.48
2,14253,3.0,Egrav6_0.2SUB,None,3.51,-25.88,-0.72,0.00,2.17,84.47
3,14254,0.0,Egrav6_0.2SUB,None,1.90,-3.90,-1.94,0.03,-1.84,84.31
4,14254,2.0,Egrav6_0.2SUB,None,1.90,-1.74,-1.94,0.03,2.04,89.94


In [9]:
# List of forces
forces = ['P', 'V2', 'V3', 'T', 'M2', 'M3']

# Dictionary to store the separate DataFrames
force_dataframes = {force: all_cs_loadcases[['Elm', 'LoadCase', force]] for force in forces}

# Access individual DataFrames
all_loadcases_P = force_dataframes['P']
all_loadcases_V2 = force_dataframes['V2']
all_loadcases_V3 = force_dataframes['V3']
all_loadcases_T = force_dataframes['T']
all_loadcases_M2 = force_dataframes['M2']
all_loadcases_M3 = force_dataframes['M3']


#### Step 3: Extract min and max force values.

Note: Keeping extraction for each force separate to check errors easily.

In [10]:
# Extracting max P values
P_max = all_loadcases_P.groupby(['Elm', 'LoadCase'])['P'].max().reset_index()
P_max.rename(columns={'P': 'max_P'}, inplace=True)

# Extracting min P values
P_min = all_loadcases_P.groupby(['Elm', 'LoadCase'])['P'].min().reset_index()
P_min.rename(columns={'P': 'min_P'}, inplace=True)

In [11]:
# Extracting max v2 values
V2_max = all_loadcases_V2.groupby(['Elm', 'LoadCase'])['V2'].max().reset_index()
V2_max.rename(columns={'V2': 'max_V2'}, inplace=True)

# Extracting min v2 values
V2_min = all_loadcases_V2.groupby(['Elm', 'LoadCase'])['V2'].min().reset_index()
V2_min.rename(columns={'V2': 'min_V2'}, inplace=True)

In [12]:
# Extracting max v3 values
V3_max = all_loadcases_V3.groupby(['Elm', 'LoadCase'])['V3'].max().reset_index()
V3_max.rename(columns={'V3': 'max_V3'}, inplace=True)

# Extracting min v3 values
V3_min = all_loadcases_V3.groupby(['Elm', 'LoadCase'])['V3'].min().reset_index()
V3_min.rename(columns={'V3': 'min_V3'}, inplace=True)

In [13]:
# Extracting max T values
T_max = all_loadcases_T.groupby(['Elm', 'LoadCase'])['T'].max().reset_index()
T_max.rename(columns={'T': 'max_T'}, inplace=True)

# Extracting min T values
T_min = all_loadcases_T.groupby(['Elm', 'LoadCase'])['T'].min().reset_index()
T_min.rename(columns={'T': 'min_T'}, inplace=True)

In [14]:
# Extracting max M2 values
M2_max = all_loadcases_M2.groupby(['Elm', 'LoadCase'])['M2'].max().reset_index()
M2_max.rename(columns={'M2': 'max_M2'}, inplace=True)

# Extracting min M2 values
M2_min = all_loadcases_M2.groupby(['Elm', 'LoadCase'])['M2'].min().reset_index()
M2_min.rename(columns={'M2': 'min_M2'}, inplace=True)

In [15]:
# Extracting max M3 values
M3_max = all_loadcases_M3.groupby(['Elm', 'LoadCase'])['M3'].max().reset_index()
M3_max.rename(columns={'M3': 'max_M3'}, inplace=True)

# Extracting min M3 values
M3_min = all_loadcases_M3.groupby(['Elm', 'LoadCase'])['M3'].min().reset_index()
M3_min.rename(columns={'M3': 'min_M3'}, inplace=True)

In [16]:
# List of DataFrames
forces_dfs = [P_max, P_min, V2_max, V2_min, V3_max, V3_min, T_max, T_min, M2_max, M2_min, M3_max, M3_min]

# Use reduce to merge all DataFrames
analysed_df = reduce(lambda left, right: pd.merge(left, right, on=['Elm', 'LoadCase'], how='inner'), forces_dfs)

#### Step 4: Add location to each of these points

In [17]:
# Convert Frame combined file to DataFrame

all_frame_path = 'C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\Location\\FRAME - Combined.xlsx'
required_cols = ['Frame', 'CentroidX', 'CentroidY']

frames = pd.read_excel(all_frame_path, usecols = required_cols)
frames = frames.rename(columns = {'Frame' : 'Elm'})

In [18]:
# Merging frames and result_df
all_elements_df = pd.merge(analysed_df, frames, on = 'Elm', how = 'left')

In [19]:
all_elements_df.shape

(9295000, 16)

In [20]:
all_elements_df.head()

,Elm,LoadCase,max_P,min_P,max_V2,min_V2,max_V3,min_V3,max_T,min_T,max_M2,min_M2,max_M3,min_M3,CentroidX,CentroidY
0,628,Egrav6_0.2SUB,2.58,2.58,-31.97,-36.54,-0.72,-0.72,0.0,0.0,2.15,0.0,102.72,0.0,81.5,159.0
1,628,Egrav7,1.10,1.10,-8.96,-10.24,-0.28,-0.28,0.0,0.0,0.85,0.0,28.79,0.0,81.5,159.0
2,628,FpF.OU8R6_0.2SUB_fEE+0.75X+2.5Y,62.31,62.31,-11.64,-16.26,-15.60,-15.60,0.0,0.0,46.79,0.0,41.81,0.0,81.5,159.0
3,628,FpF.OU8R6_0.2SUB_fEE+0.75X+2.5Yn,48.76,48.76,-6.86,-11.48,-14.87,-14.87,0.0,0.0,44.60,0.0,27.48,0.0,81.5,159.0
4,628,FpF.OU8R6_0.2SUB_fEE+0.75X+2.5Yp,75.87,75.87,-16.41,-21.04,-16.33,-16.33,0.0,0.0,48.98,0.0,56.14,0.0,81.5,159.0


In [21]:
# Convert to Parquet (data format)
all_elements_df.to_parquet('C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\all_elements_analyzed.parquet', engine = 'pyarrow', index = False)